In [4]:
import shelve
import os
import pickle
import spacy

index_fn = "/data/assertion_shelve_wiki"
max_len = 50

#nlp = spacy.load('en', disable=['parser', 'ner', 'textcat'])
nlp = spacy.load('en', parser=False, entity=False, matcher=False)

In [5]:
object2assertions = dict()
subject2assertions = dict()
assertions = dict()

def lemmatized(tokens, start, end):
    return " ".join(t.lemma_ for t in tokens if t.idx < end and t.idx >= start)


ctr = 0

def process_line(l, source):
    global ctr
    #e.g.:  'pair of compasses synonym compass\t0:17\t26:33'
    try:
        l = l.strip()
        if "\t" not in l:
            return

        [text, subject_spans, object_spans] = l.split("\t")
        
        if text.count(" ") < max_len:
            tokens = nlp(text)
            subjects = [lemmatized(tokens, int(s[:s.index(":")]), int(s[s.index(":") + 1:]))
                        for s in subject_spans.split(",")]
            objects = [lemmatized(tokens, int(s[:s.index(":")]), int(s[s.index(":") + 1:]))
                       for s in object_spans.split(",")]

            if len(subjects) == 1 and len(objects) == 1 and subjects[0] == objects[0]:
                return

            ctr += 1

            for subject in subjects:
                if subject not in subject2assertions[source]:
                    subject2assertions[source][subject] = set()
                subject2assertions[source][subject].add(str(ctr))

            for obj in objects:
                if obj not in object2assertions[source]:
                    object2assertions[source][obj] = set()
                object2assertions[source][obj].add(str(ctr))
            assertions[str(ctr)] = text
        #else:
            #print("skipping assertion bigger than %d..." % max_len)
    except ValueError:
        print("Could not process line: " + l)

In [6]:
if os.path.exists(index_fn):
    import shutil
    shutil.rmtree(index_fn)
    
os.mkdir(index_fn)  


def process_source(name, path):
    global assertions, ctr
    print("Processing %s assertions..." % name)
    object2assertions[name] = dict()
    subject2assertions[name] = dict()
    with open(path) as f:
        for l in f:
            process_line(l, name)
            if ctr % 100000 == 0:
                print("%d" % ctr)
                db.update(assertions)
                del assertions
                assertions = dict()

with shelve.open(os.path.join(index_fn, 'assertions.shelve')) as db:
    #process_source("dbpedia_type", "/run/media/diwe01/Data3/wiki/dbpedia/type_assertions.txt") 
    #process_source("microsoft_type", "/run/media/diwe01/Data2/corpora/concepts/concepts_microsoft/data-concept/type_assertions.txt") 
    #process_source("simple_wikipedia_firstsent", "/run/media/diwe01/Data3/wiki/simple_abstracts/wiki_assertions.txt") 
    #process_source("en_wikt", "/run/media/diwe01/Data3/wiki/en_wikt/meaning_assertions.txt") 
    process_source("wikipedia_firstsent", "/run/media/diwe01/Data3/wiki/en_abstracts/wiki_assertions.txt") 
    #process_source("ppdb_L", "/run/media/diwe01/Data2/corpora/concepts/ppdb/assertions_pos.txt") 
    #process_source("conceptnet", "/run/media/diwe01/Data2/corpora/concepts/conceptnet/assertions.txt")

    db.update(assertions)
    

with open(os.path.join(index_fn, 'subject2assertions.pkl'), "wb") as f:
    pickle.dump(subject2assertions, f, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(os.path.join(index_fn, 'object2assertions.pkl'), "wb") as f:
    pickle.dump(object2assertions, f, protocol=pickle.HIGHEST_PROTOCOL)

Processing wikipedia_firstsent assertions...
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000


In [7]:
import os
os.chdir('..')

from projects.assertion_mr.shared import AssertionStore

/usr/lib64/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [9]:
a_store = AssertionStore(index_fn, ["wikipedia_firstsent"])

In [10]:
from jack.io.load import load_squad

ds = load_squad('data/SQuAD/dev-v1.1.json')

In [49]:
q, answers = ds[500][0], ds[500][1]

In [41]:
from jack.util import preprocessing
from jack.util.vocab import Vocab

v = Vocab()

In [50]:
qts, _, _, qls, _ = preprocessing.nlp_preprocess(q.question, v, use_spacy=True, with_lemmas=True)
sts, _, _, sls, _ = preprocessing.nlp_preprocess(q.support[0], v, use_spacy=True, with_lemmas=True)

In [51]:
ass = a_store.get_assertion_keys(qls, sls)

In [52]:
print(q.question)
print(q.support[0])
print('A:', answers[0].text)
print()
for a in ass[0]:
    print(ass[0][a], qts[ass[1][a][0][0]:ass[1][a][0][1]], '----', sts[ass[1][a][1][0]:ass[1][a][1][1]])
    print(a_store.get_assertion(a))
    print()

What was media day called for Super Bowl 50?
The game's media day, which was typically held on the Tuesday afternoon prior to the game, was moved to the Monday evening and re-branded as Super Bowl Opening Night. The event was held on February 1, 2016 at SAP Center in San Jose. Alongside the traditional media availabilities, the event featured an opening ceremony with player introductions on a replica of the Golden Gate Bridge.
A: Super Bowl Opening Night.

6.688068485821294e-05 ['day'] ---- ['Tuesday']
tuesday is a day of the week occurring after monday and before wednesday.

6.688068485821294e-05 ['day'] ---- ['Tuesday']
tuesday is a day of the week.

7.643506840938621e-05 ['day'] ---- ['afternoon']
afternoon is that time of day from 1:00 pm and 5:00 pm.afternoon may also refer to:

8.917424647761726e-05 ['day'] ---- ['evening']
evening is the period of time near the end of the day, usually from 6:00pm to nighttime.

1.1889899530348968e-05 ['day'] ---- ['Night']
night is day is a 2012

In [29]:
ass[1][a]

([2, 3], [102, 103])